In [1]:
from PIL import Image;
import numpy as np;
import operator
import matplotlib.pyplot as plt;
import random;
import math;
import statistics

In [2]:
D = (255**2)*3;

def dist(PIX_A, PIX_B):
    d = (PIX_A[0] - PIX_B[0])**2 + (PIX_A[1] - PIX_B[1])**2 + (PIX_A[2] - PIX_B[2])**2;
    return math.sqrt(d/D);

In [3]:
TC = "004"

In [4]:
source_dir = "Source_Images/source_" + TC + ".JPG"
source_image = Image.open(source_dir)
source_pix = source_image.load()

In [5]:
test_dir = "Copied_Images/copy_" + TC + ".JPG"
test_image = Image.open(test_dir)
test_pix = test_image.load()

In [6]:
source_image.size, test_image.size

((1413, 959), (630, 402))

In [7]:
# spread = 5;
# for k in range(1, spread+1, 1):
#     for l in range(0, k, 1):
#         A = l;
#         B = k - l;
#         print(A, B, A, -B, -B, -A, B, -A)

In [8]:
spread = 1;

matrix = np.zeros((test_image.size[0], test_image.size[1]))
start_points = []

for i in range(0, test_image.size[0], 1):
    for j in range(0, test_image.size[1], 1):
        dissimilarity = 0;
        ct = 0;
        for k in range(i-spread, i+spread+1, 1):
            for l in range(j-spread, j+spread+1, 1):
                if(k == i and j == l):
                    continue;
                if(k < 0 or k >= test_image.size[0] or l < 0 or l >= test_image.size[1]):
                    continue;
                dissimilarity += dist(test_pix[k, l], test_pix[i, j]);
        
#         Diss = dissimilarity/ct;
        matrix[i][j] = dissimilarity * statistics.variance(test_pix[i, j]);
        
        start_points.append([dissimilarity, i, j]);

start_points = sorted(start_points, key = operator.itemgetter(0));
start_points.reverse()

In [9]:
len(start_points)

253260

In [10]:
LMT = 0;
MARK_POINTS = math.floor((len(start_points)*10)/100)
print(MARK_POINTS)
for i in range(0, MARK_POINTS, 1):
    X = start_points[i][1];
    Y = start_points[i][2];
    for j in range(X-LMT, X+LMT+1, 1):
        for k in range(Y - LMT, Y + LMT+1, 1):
            if(j < 0 or j >= test_image.size[0] or k < 0 or k >= test_image.size[1]):
                continue;
            test_pix[j, k] = (0, 0, 0);

test_image.save("W.jpg")

25326


In [11]:
vibrant_points = start_points[:MARK_POINTS]

In [12]:
class Path:
    def __init__(self):
        self.points = [];
        self.score = [];
        self.agg_score = -1;

In [13]:
def aggregate_score(scores):
    return statistics.median(scores)
    random.shuffle(scores);
    total_buckets = 100;
    max_bucket_size = len(scores)/total_buckets;
    buckets = [];
    curr_size = 0;
    curr_sum = 0;
    for sc in scores:
        curr_size += 1;
        curr_sum += sc;
        if(curr_size == max_bucket_size):
            buckets.append(curr_sum/curr_size);
            curr_sum = 0;
            curr_size = 0;
    if(curr_size > 0):
        buckets.append(curr_sum/curr_size);
    
    return statistics.median(buckets);

In [14]:
ans = Path();
ans.agg_score = -1;

In [15]:
# MAX_STEP = 10;
# MAX_STEP = math.floor(min(test_image.size[0], test_image.size[1])/10);

In [16]:
# RANDOM_WALK_LEN = 10000;

In [17]:
# MAX_STEP

In [18]:
statistics.mean(source_pix[0, 1]), source_pix[0, 1], statistics.stdev(source_pix[0, 1])

(99, (98, 97, 102), 2.6457513110645907)

In [19]:
source_points = [];

for i in range(0, source_image.size[0], 1):
    for j in range(0, source_image.size[1], 1):
        mn = statistics.mean(source_pix[i, j]);
        mn = round(mn, 2)
        source_points.append([mn, i, j]);

source_points = sorted(source_points, key = operator.itemgetter(0));

In [20]:
len(source_points)

1355067

In [21]:
def get_start_points(mn):
    mn = round(mn, 2)
    lower_lmt = round(mn - (mn * 20/100), 2);
    upper_lmt = round(mn + (mn * 20/100), 2);
    
    A, B = len(source_points), -1;
    l, r = 0, len(source_points)-1;
    while(l <= r):
        mid = math.floor(l + (r - l)/2);
        source_point_mn = source_points[mid][0];
        if(source_point_mn >= lower_lmt and source_point_mn <= upper_lmt):
            A = mid;
            r = mid-1;
        elif(source_point_mn < lower_lmt):
            l = mid+1;
        else:
            r = mid-1;
    
    l, r = 0, len(source_points)-1;
    while(l <= r):
        mid = math.floor(l + (r - l)/2);
        source_point_mn = source_points[mid][0];
        if(source_point_mn >= lower_lmt and source_point_mn <= upper_lmt):
            B = mid;
            l = mid+1;
        elif(source_point_mn < lower_lmt):
            l = mid+1;
        else:
            r = mid-1;
    
#     print(lower_lmt, upper_lmt)
#     print(A, B)
    return source_points[A: B+1];

source_start_points = get_start_points(vibrant_points[0][0]);

In [22]:
len(source_start_points), len(vibrant_points)

(1572, 25326)

In [23]:
# vibrant_points[0]

In [24]:
paths = [];

for i in range(0, len(source_start_points), 1):
    dis = dist(source_pix[source_start_points[i][1], source_start_points[i][2]], 
               test_pix[vibrant_points[0][1], vibrant_points[0][2]])
    p = Path();
    p.points.append([source_start_points[i][1], source_start_points[i][2]]);
    p.score.append(dis);
    paths.append(p);

In [25]:
paths[1370].score

[0.023529411764705882]

In [26]:
for i in range(1, 10000, 1):
    
    X_offset_vib = vibrant_points[0][1] - vibrant_points[i-1][1];
    Y_offset_vib = vibrant_points[0][2] - vibrant_points[i-1][2];
    
    for j in range(0, len(paths), 1):
        if(paths[j].score[-1] == -1):
            continue;
            
        prev_X = paths[0].points[-1][0];
        prev_Y = paths[0].points[-1][1];
        
        new_X = prev_X + X_offset_vib;
        new_Y = prev_Y + Y_offset_vib;
        
        if(new_X >= source_image.size[0] or new_X < 0 or new_Y < 0 or new_Y >= source_image.size[1]):
            paths[j].score.append(-1);
            continue;
        
        dis = dist(source_pix[new_X, new_Y], test_pix[vibrant_points[i][1], vibrant_points[i][2]]);
        paths[j].points.append([new_X, new_Y]);
        paths[j].score.append(dis);

In [27]:
for i in range(0, len(paths), 1):
    if(paths[i].score[-1] == -1):
        continue;
    paths[i].agg_score = aggregate_score(paths[i].score);
    

In [28]:
ans = Path();
ans.agg_score = -1;

for i in range(0, len(paths), 1):
    if(paths[i].score[-1] == -1):
        continue;
    
    if(ans.agg_score == -1 or ans.agg_score > paths[i].agg_score):
        ans = paths[i]
        

In [29]:
ans.agg_score

-1

In [30]:
output_dir = "Results/result_" + TC + ".jpg";
for i in range(0, len(ans.points), 1):
#     print(final_P.points[i]);
    source_pix[ans.points[i][0], ans.points[i][1]] = (0, 0, 0)

source_image.save(output_dir)